##### Copyright 2018 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Cómo usar una GPU

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/gpu"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/gpu.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/gpu.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/gpu.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>

El código de TensorFlow y los modelos `tf.keras` se ejecutarán de forma transparente en una única GPU sin necesidad de modificar el código.

Nota: Use `tf.config.list_physical_devices('GPU')` para confirmar que TensorFlow esté usando la GPU.

La forma más sencilla de ejecutar en múltiples GPU, en una o varias máquinas, es usar [Estrategias de distribución](distributed_training.ipynb).

Esta guía es para usuarios que han probado estos planteamientos y han descubierto que necesitan controlar con precisión la forma en que TensorFlow utiliza la GPU. Para aprender a depurar problemas de rendimiento en escenarios con una o varias GPU, consulte la guía [Optimice el rendimiento de la GPU de TensorFlow](gpu_performance_analysis.md).

## Preparación

Asegúrese de tener instalada la última versión de GPU de TensorFlow.

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## Descripción general


TensorFlow admite la ejecución de computaciones en una variedad de tipos de dispositivos, incluidos CPU y GPU. Estos se representan con identificadores de cadena, por ejemplo:

- `"/device:CPU:0"`: la CPU de su máquina.
- `"/GPU:0"`: anotación abreviada de la primera GPU de su máquina que es visible para TensorFlow.
- `"/job:localhost/replica:0/task:0/device:GPU:1"`: nombre completamente cualificado de la segunda GPU de su máquina que es visible para TensorFlow.

Si una operación de TensorFlow tiene implementaciones tanto de CPU como de GPU, por defecto, el dispositivo GPU tiene prioridad a la hora de asignar la operación. Por ejemplo, `tf.matmul` tiene tanto núcleos de CPU como de GPU y, en un sistema con dispositivos `CPU:0` y `GPU:0`, se selecciona el dispositivo `GPU:0` para ejecutar `tf.matmul` a menos que le solicite explícitamente que se ejecute en otro dispositivo.

Si una operación de TensorFlow no tiene una implementación de GPU correspondiente, entonces la operación vuelve al dispositivo CPU. Por ejemplo, dado que `tf.cast` solo tiene un núcleo de CPU, en un sistema con dispositivos `CPU:0` y `GPU:0`, se selecciona el dispositivo `CPU:0` para ejecutar `tf.cast`, incluso si se le solicita que se ejecute en el dispositivo `GPU:0`.

## Cómo registrar la colocación en el dispositivo

Para saber a qué dispositivos se asignan tus operaciones y tensores, introduce `tf.debugging.set_log_device_placement(True)` como primera instrucción de tu programa. Activar el registro de colocación en el dispositivo hace que se impriman todas las asignaciones u operaciones de los tensores.

In [ ]:
tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

El código de arriba imprimirá una indicación de que la operación `MatMul` se ejecutó en `GPU:0`.

## Colocación manual en el dispositivo

Si desea que una operación específica se ejecute en un dispositivo de su elección en lugar del que se selecciona automáticamente, puede usar `with tf.device` para crear un contexto de dispositivo, y todas las operaciones dentro de ese contexto se ejecutarán en el mismo dispositivo designado.

In [ ]:
tf.debugging.set_log_device_placement(True)

# Place tensors on the CPU
with tf.device('/CPU:0'):
  a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

# Run on the GPU
c = tf.matmul(a, b)
print(c)

Notará que ahora se asignan los identificadores `a` y `b` a `CPU:0`. Como no se especificó explícitamente un dispositivo para la operación `MatMul`, el tiempo de ejecución de TensorFlow elegirá uno en función de la operación y los dispositivos disponibles (`GPU:0` en este ejemplo) y, si es necesario, copiará automáticamente los tensores entre dispositivos.

## Cómo limitar el crecimiento de la memoria de la GPU

De forma predeterminada, TensorFlow asigna casi toda la memoria de la GPU de todas las GPU (sujetas a [`CUDA_VISIBLE_DEVICES`](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars)) visibles al proceso. Esto se hace para hacer un uso más eficiente de los recursos de memoria relativamente valiosos de la GPU en los dispositivos mediante la reducción de la fragmentación de la memoria. Para limitar TensorFlow a un conjunto específico de GPU, use el método `tf.config.set_visible_devices`.

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

En algunos casos resulta conveniente que el proceso asigne solo un subconjunto de la memoria disponible, o que solo aumente el uso de memoria a medida que lo requiera el proceso. TensorFlow ofrece dos métodos para controlar esto.

La primera consiste en activar el crecimiento de la memoria llamando a `tf.config.experimental.set_memory_growth`, que intenta asignar únicamente tanta memoria de la GPU como sea necesaria para las asignaciones en tiempo de ejecución: al principio, asigna muy poca memoria y, a medida que el programa se ejecuta y se necesita más memoria de la GPU, la región de memoria de la GPU se amplía para el proceso de TensorFlow. La memoria no se libera, ya que esto podría provocar su fragmentación. Para activar el crecimiento de memoria para una GPU específica, use el siguiente código antes de asignar un tensor o ejecutar una operación.

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

Otra forma de activar esta opción consiste en establecer la variable de entorno `TF_FORCE_GPU_ALLOW_GROWTH` en `true`. Esta configuración es específica de cada plataforma.

La segunda opción consiste en configurar un dispositivo GPU virtual con `tf.config.set_logical_device_configuration` y establecer un límite estricto para la memoria total que se asignará a la GPU.

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

Esto resulta útil si desea limitar realmente la cantidad de memoria de la GPU disponible para el proceso de TensorFlow. Esta práctica es habitual en el desarrollo local cuando la GPU se comparte con otras aplicaciones, como la interfaz gráfica de una estación de trabajo.

## Cómo utilizar una sola GPU en un sistema con múltiples GPU

Si tienes más de una GPU en tu sistema, se seleccionará por defecto la GPU con el ID más bajo. Si desea que se ejecute en otra GPU, tendrá que especificar esta preferencia de forma explícita:

In [ ]:
tf.debugging.set_log_device_placement(True)

try:
  # Specify an invalid GPU device
  with tf.device('/device:GPU:2'):
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    c = tf.matmul(a, b)
except RuntimeError as e:
  print(e)

Si el dispositivo que especificó no existe, obtendrá el mensaje `RuntimeError`: `.../device:GPU:2 unknown device`.

Si quiere que TensorFlow elija automáticamente un dispositivo existente y compatible para ejecutar las operaciones en caso de que el dispositivo especificado no exista, puede llamar `tf.config.set_soft_device_placement(True)`.

In [ ]:
tf.config.set_soft_device_placement(True)
tf.debugging.set_log_device_placement(True)

# Creates some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

## Cómo usar múltiples GPU

Desarrollar para múltiples GPU permite escalar un modelo con los recursos adicionales. Si se desarrolla en un sistema con una sola GPU, es posible emular varias GPU con dispositivos virtuales. Esto permite probar fácilmente configuraciones de múltiples GPU sin necesidad de contar con recursos adicionales.

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Create 2 virtual GPUs with 1GB memory each
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=1024),
         tf.config.LogicalDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

Una vez que el tiempo de ejecución disponga de varias GPU lógicas, puede utilizarlas con `tf.distribute.Strategy` o con colocación manual.

#### Con `tf.distribute.Strategy`

La mejor práctica para usar múltiples GPU consiste en usar `tf.distribute.Strategy`. Veamos un ejemplo sencillo:

In [ ]:
tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)
with strategy.scope():
  inputs = tf.keras.layers.Input(shape=(1,))
  predictions = tf.keras.layers.Dense(1)(inputs)
  model = tf.keras.models.Model(inputs=inputs, outputs=predictions)
  model.compile(loss='mse',
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.2))

Este programa ejecutará una copia de su modelo en cada GPU, dividiendo los datos de entrada entre ellas, lo que también se conoce como "[paralelismo de datos](https://en.wikipedia.org/wiki/Data_parallelism)".

Para obtener más información sobre estrategias de distribución, consulte la guía [aquí](./distributed_training.ipynb).

#### Colocación manual

`tf.distribute.Strategy` funciona de forma interna al replicar la computación entre dispositivos. Puede implementar la replicación manualmente si construye su modelo en cada GPU. Por ejemplo:

In [ ]:
tf.debugging.set_log_device_placement(True)

gpus = tf.config.list_logical_devices('GPU')
if gpus:
  # Replicate your computation on multiple GPUs
  c = []
  for gpu in gpus:
    with tf.device(gpu.name):
      a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
      b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
      c.append(tf.matmul(a, b))

  with tf.device('/CPU:0'):
    matmul_sum = tf.add_n(c)

  print(matmul_sum)